In [2]:
import datetime
import pandas as pd
import gymnasium as gym
import gym_anytrading
import numpy as np
import datetime
from gym_anytrading.envs import StocksEnv
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam
import subprocess

2024-01-08 13:18:12.089663: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = gym_anytrading.datasets.STOCKS_GOOGL.copy()
df.shape

(2335, 6)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2335 entries, 2009-05-22 to 2018-08-29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2335 non-null   float64
 1   High       2335 non-null   float64
 2   Low        2335 non-null   float64
 3   Close      2335 non-null   float64
 4   Adj Close  2335 non-null   float64
 5   Volume     2335 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 127.7 KB


In [5]:
from finta import TA
df['TEMA'] = TA.TEMA(df, 12) # Triple Exponential Moving Average --> in ce directie se indreapta pretul pe parcursul a 12 zile cu accent mare pe ultimele zile
df['ER'] = TA.ER(df) # Kaufman Efficiency Indicator --> confirmare a trendului (-1,1) : -1 downtrend, 1 uptrend, aprox 0 random (trend confirmation)
df['RSI'] = TA.RSI(df) # Relative Strength Index --> intre 0 si 100, 30-70 intervalul de interes, 30-0 oversold, 70-100 overbought (trend confirmation)
df['OBV'] = TA.OBV(df) # On Balance Volume --> confirmare a trendului (trend confirmation) --> se bazeaza pe volumul tranzactiilor
df['STOCH'] = TA.STOCH(df) # Stochastic Oscillator --> confirmare a trendului (trend confirmation) --> intre 0 si 100, 20-80 intervalul de interes, 20-0 oversold, 80-100 overbought
df.fillna(0, inplace=True)
df.head(20)

,Open,High,Low,Close,Adj Close,Volume,TEMA,ER,RSI,OBV,STOCH
Date,,,,,,,,,,,
2009-05-22,198.528534,199.524521,196.196198,196.946945,196.946945,3433700,196.946945,0.000000,0.000000,0.0,0.000000
2009-05-26,196.171173,202.702698,195.195190,202.382385,202.382385,6202700,201.859051,0.000000,100.000000,6202700.0,0.000000
2009-05-27,203.023026,206.136139,202.607605,202.982986,202.982986,6062500,203.160949,0.000000,100.000000,12265200.0,0.000000
2009-05-28,204.544540,206.016022,202.507507,205.405411,205.405411,5332200,205.100062,0.000000,100.000000,17597400.0,0.000000
2009-05-29,206.261261,208.823822,205.555557,208.823822,208.823822,5291100,207.893821,0.000000,100.000000,22888500.0,0.000000
2009-06-01,209.574570,215.015015,209.474472,213.493500,213.493500,6638100,211.763625,0.000000,100.000000,29526600.0,0.000000
2009-06-02,213.338333,215.195190,211.911911,214.414413,214.414413,5241900,214.155646,0.000000,100.000000,34768500.0,0.000000
2009-06-03,213.213211,216.446442,212.212219,216.041046,216.041046,7058500,216.149464,0.000000,100.000000,41827000.0,0.000000
2009-06-04,217.867874,220.840836,217.467468,220.360367,220.360367,7268900,219.240719,0.000000,100.000000,49095900.0,0.000000


In [6]:
# from reccurent_dqn import Agent, Utils
# from environment import CustomTradingEnv
# agent = Agent(possible_actions=[0,1],starting_mem_len=100000,max_mem_len=400000,starting_epsilon = 1, learn_rate = .0005)
# env = CustomTradingEnv(df=df,window_size=12,frame_bound=(12,1900))
# utils = Utils()
# last_100_avg = [-21]
# scores = deque(maxlen = 100)
# max_score = -float('inf')

In [7]:
# import time
# import matplotlib.pyplot as plt
# import tqdm
# env.reset()
# for i in tqdm.tqdm(range(1000)):
#     frames = [] # Saving the frames for the gif
#     timesteps = agent.total_timesteps
#     timee = time.time()
#     score,info = utils.play_episode( env, agent,i, debug = True) #set debug to true for rendering
#     print(info)
#     print('\nEpisode: ' + str(i))
#     print('Epsilon: ' + str(agent.epsilon))

In [25]:
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
from environment import CustomTradingEnv
import torch
torch.set_num_threads(1)
env = CustomTradingEnv(df=df,window_size=12,frame_bound=(12,int(len(df)*.8)),balance=1000000)
print(env._get_info())
vectorized_env = DummyVecEnv([lambda: env])
model = RecurrentPPO('MlpLstmPolicy', vectorized_env,  verbose=0,
                    gamma=0.9681627173784817,
                    learning_rate= 0.29392269361503504,
                    policy_kwargs={
                    'enable_critic_lstm': False,
                    'lstm_hidden_size': 256, 
                    'n_lstm_layers': 5},
                    seed=42
                    )

# model = RecurrentPPO('MlpLstmPolicy', vectorized_env,  verbose=0,)

{'total_reward': None, 'total_profit': None, 'position': None, 'balance': 1000000}

In [24]:
model.learn(total_timesteps=int(2e4),progress_bar=True)

Output()

KeyboardInterrupt: 

In [21]:
from environment import CustomTradingEnv
from tqdm import tqdm
def evaluate_agent(df,model,episodes):
    test_env = CustomTradingEnv(df=df, frame_bound=(int(0.8*len(df)), len(df)), window_size=12,balance=10000000)
    mean_reward, mean_profit,mean_balance = 0, 0,0
    for episode in tqdm(range(episodes)):
        state = test_env.reset()[0]
        while True:
            action, x = model.predict(state)
            state, reward, deprecated,done, info = test_env.step(action)
            if done:
                print(info)
                break
        mean_profit += info['total_profit']
        mean_reward += info['total_reward']
        mean_balance += info['balance']
        
    mean_reward /= episodes
    mean_profit /= episodes
    mean_balance /= episodes
    
    return mean_reward, mean_profit,mean_balance

evaluate_agent(df,model,episodes=50)

  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:00<00:47,  1.03it/s]

{'total_reward': -428.910034, 'total_profit': 1.0, 'position': <Positions.Short: 0>, 'balance': 10458021.292937582}


  4%|▍         | 2/50 [00:01<00:46,  1.02it/s]

{'total_reward': -428.910034, 'total_profit': 1.0, 'position': <Positions.Short: 0>, 'balance': 10458021.292937582}


...

  8%|▊         | 4/50 [00:04<00:49,  1.08s/it]

{'total_reward': -428.910034, 'total_profit': 1.0, 'position': <Positions.Short: 0>, 'balance': 10458021.292937582}


 10%|█         | 5/50 [00:05<00:48,  1.08s/it]

{'total_reward': -428.910034, 'total_profit': 1.0, 'position': <Positions.Short: 0>, 'balance': 10458021.292937582}


KeyboardInterrupt: 

In [19]:
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
from environment import CustomTradingEnv

env = CustomTradingEnv(df=df,window_size=12,frame_bound=(12,int(len(df)*.8)))
vectorized_env = DummyVecEnv([lambda: env])
model_cnn_lstm = RecurrentPPO('CnnLstmPolicy', vectorized_env,  verbose=0)

535038.9905027788

In [ ]:
model_cnn_lstm.learn(total_timesteps=int(2e4),progress_bar=True)